In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
from datetime import date

from sklearn.linear_model import  LogisticRegression,LinearRegression
from sklearn.metrics import accuracy_score

from sklearn import datasets
from sklearn.metrics import roc_curve, auc
import statsmodels.api as sm
import sklearn

from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_california_housing
from sklearn.neighbors import BallTree
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv('../ml/data/portals_numeric_estates.csv', low_memory = False)

In [4]:
df2=df[(df.main_type_id == 2)]

In [5]:
df3 = df2.drop([
    "Unnamed: 0",
    "main_type_id"], axis=1)

In [6]:
df3 = df3.drop([
    "fixed_city",
    "pop_province"], axis=1)

In [7]:
df3.apartment_floor.fillna(0,inplace=True)
#jeżeli 'apartment_room' to liczba pokoi a nie wszystkich pomieszczeń
df3.apartment_room_number.fillna(1,inplace=True)


In [8]:
ilo=df3.building_year.isnull().sum()/df3.building_year.sum()
ilo
#stanowi to 0.5% więc można usunąc te wyniki bądź spróbować znaleźć, poprzez kmean albo inny model jaki powinen być wiek budynku
#na ten moment ja wyrzucam
df4 = df3[df3.building_year.notnull()]
# analiza tylko tych obiektów, które posiadają cene
df5 = df4[df4.price.notnull()]

In [9]:
df5.dropna(inplace=True)

In [10]:
df5.dropna(inplace=True)
#df5.columns

In [11]:
def rm_out(df,columns=None):
    if columns:
        columns = columns
    else:
        columns = df.columns
        
    df_out = df.copy()
    for c in columns:
        q1 = df[c].quantile(0.25)
        q3 = df[c].quantile(0.75)
        IRQ = q3 - q1
        df_out = df_out.loc[ (df[c] <= q3 + 1.5 * IRQ) & (df[c] >= q1 - 1.5 * IRQ) ]
        
    df_out.index = range(len(df_out))
    return df_out
df6=rm_out(df5)
#df5.groupby(['building_year']).count()

In [12]:
df7 = df6.drop(["price"], axis=1)

In [13]:
train_X, test_X  = train_test_split(df7,  random_state=101,test_size =0.1)
##------------ tworzenie obiektu do wyznaczania najbliższych punktów w przestrzeni ---
scalar = MinMaxScaler().fit(train_X)
train_X_norm = scalar.transform(train_X)
test_X_norm = scalar.transform(test_X)
tree = BallTree(train_X_norm, leaf_size=100,metric ='euclidean') 


In [14]:
#K = 3
test_index = 51

dist, ind = tree.query(test_X_norm, k=5)     
print(test_X.iloc[test_index])

area_total                   47.450000
apartment_floor               4.000000
apartment_room_number         3.000000
building_year              2020.000000
latitude                     50.270894
longitude                    19.000967
pop_population           295449.000000
pop_woman                154964.000000
pop_man                  140485.000000
Name: 78268, dtype: float64


In [15]:
#print('odległości k najbliższych sąsiadów od wybranej obserwacji',dist[test_index])
#print('identyfikatory k najbliższych sąsiadów ',ind[test_index])
# ---- parametry najbliższych sąsiadów ---
#train_X.iloc[ind[test_index]]

In [16]:
# Wylistowanie podobnych mieskzań
wynik = pd.DataFrame(columns = ['Cena','Powierzchnia całkowita','Cena za m2','Liczba pokoi','Piętro' ,'Rok budowy'])
for i in ind[test_index]:
    id=train_X.index[i]
    price=float(df6.price[id])
    area_total=round(df6.area_total[id],1)
    price_per_m = round(df6.price[id]/df6.area_total[id],1)
    apartment_room_number=df6.apartment_room_number[id]
    apartment_floor=df6.apartment_floor[id]
    building_year=df6.building_year[id]
    wynik = pd.DataFrame(np.array([[price, area_total,price_per_m,apartment_room_number,apartment_floor, building_year]]),columns=['Cena','Powierzchnia całkowita','Cena za m2','Liczba pokoi','Piętro' ,'Rok budowy']).append(wynik, ignore_index=True)
wynik

,Cena,Powierzchnia całkowita,Cena za m2,Liczba pokoi,Piętro,Rok budowy
0,257200.0,39.0,6593.2,3.0,4.0,2020.0
1,371283.0,52.6,7061.3,3.0,4.0,2020.0
2,305900.0,50.8,6016.9,3.0,4.0,2020.0
3,305900.0,50.8,6016.9,3.0,4.0,2020.0
4,379000.0,47.4,7987.4,3.0,4.0,2020.0


In [17]:
#Wycena mieszkania
print('Wycena mieszkania na podstawie mieszkań podobnych:')
print('1. Wg cena całkowitych mieszkań podobnych')
print(" - cena średnia:", pd.to_numeric(wynik['Cena'], errors='coerce').apply(float).mean())
print(" - cena minimalna: ", pd.to_numeric(wynik['Cena'], errors='coerce').apply(float).min())
print(" - cena maksymalna: ", pd.to_numeric(wynik['Cena'], errors='coerce').apply(float).max())
print('2. Wg cen za m2 mieszkań podobnych')
print(" - średnia cena za m2:", pd.to_numeric(wynik['Cena za m2'], errors='coerce').apply(float).mean())
print(" - cena minimalna za m2: ", pd.to_numeric(wynik['Cena za m2'], errors='coerce').apply(float).min())
print(" - cena maksymalna za m2: ", pd.to_numeric(wynik['Cena za m2'], errors='coerce').apply(float).max())
print("3. Wycena mieszkania testowego na podstawie średniej ceny mieszkań podobnych: ",round(test_X.iloc[test_index]['area_total']*pd.to_numeric(wynik['Cena za m2'], errors='coerce').apply(float).mean(),1) )
print("4. Cena mieszkania testowego w ofercie: ", df6.price[test_index])

Wycena mieszkania na podstawie mieszkań podobnych:
1. Wg cena całkowitych mieszkań podobnych
 - cena średnia: 323856.6
 - cena minimalna:  257200.0
 - cena maksymalna:  379000.0
2. Wg cen za m2 mieszkań podobnych
 - średnia cena za m2: 6735.140000000001
 - cena minimalna za m2:  6016.9
 - cena maksymalna za m2:  7987.4
3. Wycena mieszkania testowego na podstawie średniej ceny mieszkań podobnych:  319582.4
4. Cena mieszkania testowego w ofercie:  400000.0
